## AAVS3 station initialisation and equalisation

In [1]:
import numpy as np
import statistics
import tango
import time
from datetime import datetime, timedelta
import json
from pandas import DataFrame

Get a handle on our TPMs and make sure we are getting live data, not data cached by Tango's polling system.

In [8]:
tpms = [Device(f"low-mccs/tile/aavs3-{i:02}") for i in range(1, 17)]
subracks = [Device(f"low-mccs/subrack/aavs3-{i}") for i in range(1, 3)]
station = Device(f"low-mccs/spsstation/aavs3")
daq = Device(f"low-mccs/daqreceiver/aavs3")
for dev in tpms + subracks + [station, daq]:
    dev.set_source(tango.DevSource.DEV)
AdminMode = type(station.adminMode)

In [3]:
for tpm in tpms:
    tpm.adminMode = AdminMode.ONLINE

**FIXME** the stationId and logicalTileId attributes of MccsTile are not always correctly set by SpsStation - disabling and re-enabling it forces them to be re-set.

This is tracked in [SKB-273](https://jira.skatelescope.org/browse/SKB-273).

In [4]:
station.adminMode = AdminMode.OFFLINE
time.sleep(2)
station.adminMode = AdminMode.ONLINE

In [39]:
def tangoattr(dev, k):
    try:
        v = dev[k].value
        return str(v) if isinstance(v, tango.DevState) else v
    except Exception as e:
        return repr(e)

DataFrame({k: tangoattr(tpm, k) for k in [
    'state', 'stationId', 'logicalTileId', 'tileProgrammingState', 'fpgaTime', 'fpgaReferenceTime', 'fpgaFrameTime', 'clockPresent', 'pllLocked', 'ppsPresent', 'ppsDelay'
]} for tpm in tpms)

,state,stationId,logicalTileId,tileProgrammingState,fpgaTime,fpgaReferenceTime,fpgaFrameTime,clockPresent,pllLocked,ppsPresent,ppsDelay
0,ON,1,0,Synchronised,2024-01-24T06:13:36.000000Z,2024-01-24T06:13:32.000000Z,2024-01-24T06:13:36.195860Z,True,True,True,18
1,ON,1,1,Synchronised,2024-01-24T06:13:36.000000Z,2024-01-24T06:13:32.000000Z,2024-01-24T06:13:36.199731Z,True,True,True,18
2,ON,1,2,Synchronised,2024-01-24T06:13:36.000000Z,2024-01-24T06:13:32.000000Z,2024-01-24T06:13:36.203602Z,True,True,True,18
3,ON,1,3,Synchronised,2024-01-24T06:13:36.000000Z,2024-01-24T06:13:32.000000Z,2024-01-24T06:13:36.207473Z,True,True,True,18
4,ON,1,4,Synchronised,2024-01-24T06:13:36.000000Z,2024-01-24T06:13:32.000000Z,2024-01-24T06:13:36.211896Z,True,True,True,18
5,ON,1,5,Synchronised,2024-01-24T06:13:36.000000Z,2024-01-24T06:13:32.000000Z,2024-01-24T06:13:36.217149Z,True,True,True,18
6,ON,1,6,Synchronised,2024-01-24T06:13:36.000000Z,2024-01-24T06:13:32.000000Z,2024-01-24T06:13:36.221020Z,True,True,True,18
7,ON,1,7,Synchronised,2024-01-24T06:13:36.000000Z,2024-01-24T06:13:32.000000Z,2024-01-24T06:13:36.224891Z,True,True,True,18
8,ON,1,8,Synchronised,2024-01-24T06:13:36.000000Z,2024-01-24T06:13:32.000000Z,2024-01-24T06:13:36.229038Z,True,True,True,18
9,ON,1,9,Synchronised,2024-01-24T06:13:36.000000Z,2024-01-24T06:13:32.000000Z,2024-01-24T06:13:36.232909Z,True,True,True,17


In [9]:
import json
daq_status = json.loads(daq.DaqStatus())
tpm_config = {
    "mode": "10G",
    "destination_ip": daq_status["Receiver IP"][0],
    "destination_port": daq_status["Receiver Ports"][0],
    "channel_payload_length": 8192,
}
print(tpm_config)

{'mode': '10G', 'destination_ip': '10.137.0.111', 'destination_port': 4660, 'channel_payload_length': 8192}


Destination addresses need to be set before calling StartAcquisition. MccsTile will wait for the TPM's ARP tables to be populated before starting acquisition.

In [10]:
station.SetLmcDownload(json.dumps(tpm_config))
station.SetCspIngest(json.dumps(tpm_config))

[array([0], dtype=int32), ['SetCspIngest command completed OK']]

Initialise the station. This erases the FPGAs on each TPM, re-programs, and re-initialises them, including setting up the daisy-chaining between TPMs.

In [11]:
station.Initialise()

[array([2], dtype=int32), ['1706076699.8599222_260491772728805_Initialise']]

Set an initial flat attenuation across all ADCs before starting acquisition.

In [36]:
station.preaduLevels = [24.0] * 512

Start acquisition! Five seconds after executing this, re-execute the dataframe cell above - all TPMs should be in tileProgrammingState `Sychronised`.

In [37]:
start_time = (datetime.now().replace(microsecond=0) + timedelta(seconds=2)).strftime("%Y-%m-%dT%H:%M:%S.%fZ")
station.StartAcquisition(json.dumps({"start_time": start_time}))

[array([2], dtype=int32),
 ['1706076810.600363_124304954648635_StartAcquisition']]

First we spend a little while collecting ADC powers over time so that we aren't equalising based on an RFI spike or other transient.

In [42]:
N_SAMPLES = 20
TARGET_ADC = 17

adc_data = np.empty([N_SAMPLES, 512])
for i in range(len(adc_data)):
    time.sleep(1)
    adc_data[i] = station.adcPower

# calculate difference in dB between current and target values
adc_medians = np.median(adc_data, axis=0)
adc_deltas = 20 * np.log10(adc_medians / TARGET_ADC)

# calculate ideal attenuation
preadu_levels = np.concatenate([t.preaduLevels for t in tpms])
desired_levels = preadu_levels + adc_deltas

# quantise and clip to valid range
sanitised_levels = (desired_levels * 4).round().clip(0, 127) / 4

# apply new preADU levels to the station
station.preaduLevels = sanitised_levels

In [ ]:
adcPowers = [[list() for _ in range(32)] for _ in range(16)]
for i in range(10):
    for j, tpm in enumerate(tpms):
        for k, p in enumerate(list(tpm.adcPower)):
            adcPowers[j][k].append(p)
    time.sleep(1)

adcPowers = np.array([[statistics.mode(ch) for ch in t] for t in adcPowers])
dbmPowers = 10 * np.log10(np.power((adcPowers * (1.7 / 256.)), 2) / 400.) + 30 + 12

TARGET = -3

for i, tpm in enumerate(tpms):
    print(f"TPM {i+1}")
    # print(f"{tpm.adcPower=}")
    # dbmPower = 10 * np.log10(np.power((tpm.adcPower * (1.7 / 256.)), 2) / 400.) + 30 + 12
    dbmPower=dbmPowers[i]
    print(f"{dbmPower=}")
    newAttens = ((dbmPower + tpm.preaduLevels - TARGET) * 4).round().clip(0, 127) / 4
    attenDiff = newAttens - tpm.preaduLevels
    print(f"{attenDiff=}")
    tpm.preaduLevels = newAttens

time.sleep(2)
for i, tpm in enumerate(tpms):
    print(f"TPM {i+1}")
    print(f"{tpm.preaduLevels=}")
    dbmPower = 10 * np.log10(np.power((tpm.adcPower * (1.7 / 256.)), 2) / 400.) + 30 + 12
    print(f"{dbmPower=}\n")

    # print(tpm.adcPower)
    # print(dbmPowers[i])
    # print(10 * np.log10(np.power((tpm.adcPower * (1.7 / 256.)), 2) / 400.) + 30 + 12)